<a href="https://colab.research.google.com/github/mark76jx17/BIG-DATA-project-/blob/main/urban_services_analysis_improved_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Urban Services Analysis - Pavia & Cagliari
## Interactive 3D Visualization with H3 Spatial Indexing

Questo notebook analizza la distribuzione dei servizi essenziali (Salute, Educazione, Cibo, Sicurezza, Servizi Pubblici, Sport) nelle città di Pavia e Cagliari utilizzando dati OpenStreetMap e visualizzazione 3D interattiva.

## 1. Setup Iniziale

In [1]:
from google.colab import output
output.enable_custom_widget_manager()

## 2. Installazione Librerie

In [2]:
!pip install osmnx h3 geopandas keplergl

## 3. Import e Configurazione

In [3]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import h3
import warnings
from keplergl import KeplerGl

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="osmnx")
warnings.filterwarnings("ignore", message="Geometry is in a geographic CRS")

print("Librerie importate con successo!")

Librerie importate con successo!


## 4. Definizione Categorie e Mapping

Definiamo le categorie di servizi da analizzare e il mapping per la classificazione.

In [4]:
# Define locations
locations = ['Pavia, Italy', 'Cagliari, Italy']

# Define tags con struttura estesa e completa
tags = {
    'amenity': [
        # Health
        'hospital', 'clinic', 'doctors', 'dentist', 'pharmacy', 'veterinary',
        'nursing_home', 'social_facility', 'health_post',
        # Education
        'school', 'university', 'kindergarten', 'college', 'library',
        'music_school', 'language_school', 'driving_school', 'research_institute',
        # Food & Beverage
        'restaurant', 'cafe', 'fast_food', 'bar', 'pub', 'ice_cream',
        'food_court', 'biergarten', 'juice_bar',
        # Retail & Services
        'marketplace', 'bank', 'atm', 'bureau_de_change',
        'post_office', 'post_box', 'parcel_locker',
        # Public Services & Safety
        'police', 'fire_station', 'townhall', 'courthouse', 'community_centre',
        'social_centre', 'public_building', 'ranger_station',
        # Culture & Entertainment
        'cinema', 'theatre', 'arts_centre', 'studio', 'events_venue',
        'nightclub', 'casino', 'gambling', 'conference_centre',
        # Religion
        'place_of_worship', 'monastery',
        # Transportation
        'bicycle_parking', 'bicycle_rental', 'car_sharing', 'charging_station',
        'ferry_terminal', 'bus_station', 'taxi',
        # Other Services
        'childcare', 'toilets', 'drinking_water', 'fountain', 'waste_basket',
        'recycling', 'waste_disposal'
    ],
    'leisure': [
        # Sports
        'sports_centre', 'pitch', 'stadium', 'swimming_pool', 'fitness_centre',
        'fitness_station', 'golf_course', 'ice_rink', 'track', 'water_park',
        # Recreation
        'park', 'playground', 'garden', 'nature_reserve', 'dog_park',
        'beach_resort', 'picnic_table', 'bandstand',
        # Entertainment
        'amusement_arcade', 'bowling_alley', 'escape_game', 'hackerspace',
        'dance'
    ],
    'shop': [
        # Food Retail
        'supermarket', 'convenience', 'bakery', 'butcher', 'greengrocer',
        'seafood', 'alcohol', 'beverages', 'cheese', 'chocolate',
        'coffee', 'confectionery', 'dairy', 'deli', 'farm',
        'frozen_food', 'health_food', 'ice_cream', 'organic', 'pasta',
        'pastry', 'spices', 'tea', 'water', 'wine',
        # General Retail
        'department_store', 'general', 'kiosk', 'mall', 'wholesale',
        # Specialized Shops (essential)
        'pharmacy', 'chemist', 'baby_goods', 'medical_supply',
        'books', 'newsagent', 'stationery',
        'hardware', 'doityourself', 'trade',
        'laundry', 'dry_cleaning',
        'hairdresser', 'beauty', 'optician', 'hearing_aids'
    ],
    'healthcare': [
        'hospital', 'clinic', 'doctor', 'dentist', 'pharmacy', 'laboratory',
        'physiotherapist', 'alternative', 'audiologist', 'blood_donation',
        'midwife', 'nurse', 'occupational_therapist', 'optometrist',
        'podiatrist', 'psychotherapist', 'rehabilitation', 'speech_therapist'
    ],
    'office': [
        'government', 'administrative', 'employment_agency', 'notary',
        'lawyer', 'accountant', 'tax_advisor', 'estate_agent'
    ],
    'tourism': [
        'information', 'hotel', 'hostel', 'guest_house', 'museum',
        'gallery', 'attraction', 'viewpoint', 'picnic_site'
    ]
}

# Mapping esteso per categorizzazione
category_mapping = {
    # Health
    'hospital': 'Health', 'clinic': 'Health', 'doctors': 'Health', 'doctor': 'Health',
    'dentist': 'Health', 'pharmacy': 'Health', 'veterinary': 'Health',
    'nursing_home': 'Health', 'social_facility': 'Health', 'health_post': 'Health',
    'laboratory': 'Health', 'physiotherapist': 'Health', 'alternative': 'Health',
    'audiologist': 'Health', 'blood_donation': 'Health', 'midwife': 'Health',
    'nurse': 'Health', 'occupational_therapist': 'Health', 'optometrist': 'Health',
    'podiatrist': 'Health', 'psychotherapist': 'Health', 'rehabilitation': 'Health',
    'speech_therapist': 'Health', 'chemist': 'Health', 'medical_supply': 'Health',
    'optician': 'Health', 'hearing_aids': 'Health',

    # Education
    'school': 'Education', 'university': 'Education', 'kindergarten': 'Education',
    'college': 'Education', 'library': 'Education', 'music_school': 'Education',
    'language_school': 'Education', 'driving_school': 'Education',
    'research_institute': 'Education', 'books': 'Education', 'stationery': 'Education',

    # Food & Beverage
    'restaurant': 'Food', 'cafe': 'Food', 'fast_food': 'Food', 'bar': 'Food',
    'pub': 'Food', 'ice_cream': 'Food', 'food_court': 'Food', 'biergarten': 'Food',
    'juice_bar': 'Food',

    # Food Retail
    'supermarket': 'Food Retail', 'convenience': 'Food Retail', 'bakery': 'Food Retail',
    'butcher': 'Food Retail', 'greengrocer': 'Food Retail', 'seafood': 'Food Retail',
    'alcohol': 'Food Retail', 'beverages': 'Food Retail', 'cheese': 'Food Retail',
    'chocolate': 'Food Retail', 'coffee': 'Food Retail', 'confectionery': 'Food Retail',
    'dairy': 'Food Retail', 'deli': 'Food Retail', 'farm': 'Food Retail',
    'frozen_food': 'Food Retail', 'health_food': 'Food Retail', 'organic': 'Food Retail',
    'pasta': 'Food Retail', 'pastry': 'Food Retail', 'spices': 'Food Retail',
    'tea': 'Food Retail', 'water': 'Food Retail', 'wine': 'Food Retail',

    # Retail & Shopping
    'marketplace': 'Retail', 'department_store': 'Retail', 'general': 'Retail',
    'kiosk': 'Retail', 'mall': 'Retail', 'wholesale': 'Retail',
    'baby_goods': 'Retail', 'newsagent': 'Retail',
    'hardware': 'Retail', 'doityourself': 'Retail', 'trade': 'Retail',

    # Personal Services
    'laundry': 'Services', 'dry_cleaning': 'Services', 'hairdresser': 'Services',
    'beauty': 'Services', 'childcare': 'Services',

    # Financial Services
    'bank': 'Financial', 'atm': 'Financial', 'bureau_de_change': 'Financial',
    'accountant': 'Financial', 'tax_advisor': 'Financial',

    # Public Services
    'post_office': 'Public Services', 'post_box': 'Public Services',
    'parcel_locker': 'Public Services', 'townhall': 'Public Services',
    'courthouse': 'Public Services', 'community_centre': 'Public Services',
    'social_centre': 'Public Services', 'public_building': 'Public Services',
    'government': 'Public Services', 'administrative': 'Public Services',
    'employment_agency': 'Public Services', 'ranger_station': 'Public Services',

    # Security & Emergency
    'police': 'Security', 'fire_station': 'Security',

    # Sports & Fitness
    'sports_centre': 'Sports', 'pitch': 'Sports', 'stadium': 'Sports',
    'swimming_pool': 'Sports', 'fitness_centre': 'Sports', 'fitness_station': 'Sports',
    'golf_course': 'Sports', 'ice_rink': 'Sports', 'track': 'Sports',
    'water_park': 'Sports', 'bowling_alley': 'Sports',

    # Recreation & Parks
    'park': 'Recreation', 'playground': 'Recreation', 'garden': 'Recreation',
    'nature_reserve': 'Recreation', 'dog_park': 'Recreation', 'beach_resort': 'Recreation',
    'picnic_table': 'Recreation', 'bandstand': 'Recreation', 'picnic_site': 'Recreation',

    # Culture & Entertainment
    'cinema': 'Culture', 'theatre': 'Culture', 'arts_centre': 'Culture',
    'studio': 'Culture', 'events_venue': 'Culture', 'nightclub': 'Culture',
    'casino': 'Culture', 'gambling': 'Culture', 'conference_centre': 'Culture',
    'amusement_arcade': 'Culture', 'escape_game': 'Culture', 'hackerspace': 'Culture',
    'dance': 'Culture', 'museum': 'Culture', 'gallery': 'Culture',
    'attraction': 'Culture',

    # Religion
    'place_of_worship': 'Religion', 'monastery': 'Religion',

    # Transportation
    'bicycle_parking': 'Transportation', 'bicycle_rental': 'Transportation',
    'car_sharing': 'Transportation', 'charging_station': 'Transportation',
    'ferry_terminal': 'Transportation', 'bus_station': 'Transportation',
    'taxi': 'Transportation',

    # Tourism
    'information': 'Tourism', 'hotel': 'Tourism', 'hostel': 'Tourism',
    'guest_house': 'Tourism', 'viewpoint': 'Tourism',

    # Professional Services
    'notary': 'Professional Services', 'lawyer': 'Professional Services',
    'estate_agent': 'Professional Services',

    # Public Utilities
    'toilets': 'Public Utilities', 'drinking_water': 'Public Utilities',
    'fountain': 'Public Utilities', 'waste_basket': 'Public Utilities',
    'recycling': 'Public Utilities', 'waste_disposal': 'Public Utilities'
}

print("Configurazione completata!")
print(f"Città da analizzare: {locations}")
print(f"Categorie definite: {sorted(set(category_mapping.values()))}")
print(f"\nTotale POI types definiti: {sum(len(v) for v in tags.values())}")
print(f"Totale mappings: {len(category_mapping)}")


Configurazione completata!
Città da analizzare: ['Pavia, Italy', 'Cagliari, Italy']
Categorie definite: ['Culture', 'Education', 'Financial', 'Food', 'Food Retail', 'Health', 'Professional Services', 'Public Services', 'Public Utilities', 'Recreation', 'Religion', 'Retail', 'Security', 'Services', 'Sports', 'Tourism', 'Transportation']

Totale POI types definiti: 171
Totale mappings: 165


## 5. Download Dati OSM con Gestione Errori

Scarica i Points of Interest da OpenStreetMap per entrambe le città con gestione robusta degli errori.

In [ ]:
def download_pois_safely(locations, tags):
    """
    Download POIs with error handling
    """
    all_gdfs = []

    for location in locations:
        try:
            print(f"Downloading data for {location}...")
            gdf = ox.features.features_from_place(location, tags)

            # Aggiungi colonna città
            gdf['city'] = location.split(',')[0]
            all_gdfs.append(gdf)
            print(f"✓ {location}: {len(gdf)} POIs scaricati")

        except Exception as e:
            print(f"✗ Errore nel download di {location}: {e}")
            continue

    if all_gdfs:
        return pd.concat(all_gdfs, ignore_index=True)
    else:
        raise Exception("Nessun dato scaricato!")

# Download POIs
gdf_pois = download_pois_safely(locations, tags)

print(f"\n{'='*50}")
print(f"Totale POIs scaricati: {gdf_pois.shape[0]}")
print(f"Distribuzione per città:")
print(gdf_pois['city'].value_counts())
print(f"{'='*50}")

## 6. Categorizzazione dei Servizi

Classifica ogni POI in una delle categorie principali (Health, Education, Food, etc.).

In [ ]:
def categorize_service(row):
    """
    Determina la categoria del servizio basandosi sui tag
    Gestisce: amenity, leisure, shop, healthcare, office, tourism
    """
    # Check amenity tag
    if 'amenity' in row and pd.notna(row['amenity']):
        amenity = row['amenity']
        if amenity in category_mapping:
            return category_mapping[amenity]

    # Check leisure tag
    if 'leisure' in row and pd.notna(row['leisure']):
        leisure = row['leisure']
        if leisure in category_mapping:
            return category_mapping[leisure]

    # Check shop tag
    if 'shop' in row and pd.notna(row['shop']):
        shop = row['shop']
        if shop in category_mapping:
            return category_mapping[shop]

    # Check healthcare tag
    if 'healthcare' in row and pd.notna(row['healthcare']):
        healthcare = row['healthcare']
        if healthcare in category_mapping:
            return category_mapping[healthcare]

    # Check office tag
    if 'office' in row and pd.notna(row['office']):
        office = row['office']
        if office in category_mapping:
            return category_mapping[office]

    # Check tourism tag
    if 'tourism' in row and pd.notna(row['tourism']):
        tourism = row['tourism']
        if tourism in category_mapping:
            return category_mapping[tourism]

    return 'Other'

# Applica categorizzazione
gdf_pois['category'] = gdf_pois.apply(categorize_service, axis=1)

print("Distribuzione per categoria:")
print(gdf_pois['category'].value_counts())
print(f"\nDistribuzione per città e categoria:")
print(pd.crosstab(gdf_pois['city'], gdf_pois['category']))


## 7. Calcolo H3 Indices (Ottimizzato)

Converte le geometrie in centroids e calcola gli indici H3 (resolution 9 = celle di ~0.1 km²).

In [ ]:
# Create a copy
gdf_h3 = gdf_pois.copy()

# Convert geometries to centroids
gdf_h3['geometry'] = gdf_h3.geometry.centroid

# Extract coordinates
gdf_h3['lat'] = gdf_h3.geometry.y
gdf_h3['lng'] = gdf_h3.geometry.x

# Calculate H3 index at resolution 9 (ottimizzato)
# Resolution 9 = celle di ~0.1 km² (~174m di lato)
resolution = 9
print(f"Calcolo H3 indices (resolution {resolution})...")

# Metodo ottimizzato invece di apply
gdf_h3['h3_index'] = [
    h3.latlng_to_cell(lat, lng, resolution)
    for lat, lng in zip(gdf_h3['lat'], gdf_h3['lng'])
]

print(f"✓ H3 indices calcolati per {len(gdf_h3)} POIs")
display(gdf_h3[['city', 'category', 'lat', 'lng', 'h3_index']].head(10))

## 8. Aggregazione Multi-livello

Aggrega i dati per cella H3, calcolando il totale servizi e la distribuzione per categoria.

In [ ]:
# Aggregazione totale
df_counts_total = gdf_h3.groupby(['h3_index', 'city']).size().reset_index(name='service_count')

# Aggregazione per categoria
df_counts_category = gdf_h3.groupby(['h3_index', 'city', 'category']).size().reset_index(name='count')

# Pivot per avere una colonna per categoria
df_counts_pivot = df_counts_category.pivot_table(
    index=['h3_index', 'city'],
    columns='category',
    values='count',
    fill_value=0
).reset_index()

# Merge con totale
df_final = df_counts_total.merge(df_counts_pivot, on=['h3_index', 'city'])

# Aggiungi coordinate del centroide H3 per visualizzazione
df_final['lat'] = df_final['h3_index'].apply(lambda x: h3.cell_to_latlng(x)[0])
df_final['lng'] = df_final['h3_index'].apply(lambda x: h3.cell_to_latlng(x)[1])

print(f"Celle H3 uniche totali: {df_final.shape[0]}")
print(f"\nDistribuzione per città:")
print(df_final.groupby('city')['service_count'].describe())

## 9. Statistiche Descrittive

Analisi dettagliata della distribuzione dei servizi per città e categoria.

In [ ]:
print("="*60)
print("STATISTICHE DESCRITTIVE")
print("="*60)

for city in df_final['city'].unique():
    city_data = df_final[df_final['city'] == city]
    print(f"\n{city.upper()}")
    print("-"*60)
    print(f"Celle H3 con servizi: {len(city_data)}")
    print(f"Servizi totali: {city_data['service_count'].sum()}")
    print(f"Media servizi per cella: {city_data['service_count'].mean():.2f}")
    print(f"Mediana servizi per cella: {city_data['service_count'].median():.0f}")
    print(f"Max servizi in una cella: {city_data['service_count'].max()}")

    print(f"\nDistribuzione per categoria:")
    categories = ['Health', 'Education', 'Food', 'Food Retail', 'Retail', 'Services',
                  'Financial', 'Public Services', 'Security', 'Sports', 'Recreation',
                  'Culture', 'Religion', 'Transportation', 'Tourism', 'Professional Services',
                  'Public Utilities']
    for cat in categories:
        if cat in city_data.columns:
            total = city_data[cat].sum()
            print(f"  {cat}: {total}")

print("\n" + "="*60)


## 10. Analisi Accessibilità (15-Minute City)

Valuta quante celle sono "ben servite" secondo il concetto di città in 15 minuti.

In [ ]:
def analyze_accessibility(df, threshold_15min=3):
    """
    Analizza l'accessibilità dei servizi
    threshold_15min: numero minimo di servizi per considerare una cella "ben servita"
    (approssimazione per 15-minute city concept)
    """
    results = {}

    for city in df['city'].unique():
        city_data = df[df['city'] == city]

        well_served = len(city_data[city_data['service_count'] >= threshold_15min])
        poorly_served = len(city_data[city_data['service_count'] < threshold_15min])

        results[city] = {
            'well_served_cells': well_served,
            'poorly_served_cells': poorly_served,
            'percentage_well_served': (well_served / len(city_data) * 100) if len(city_data) > 0 else 0
        }

    return results

accessibility = analyze_accessibility(df_final)

print("ANALISI ACCESSIBILITÀ (15-Minute City Concept)")
print("="*60)
for city, stats in accessibility.items():
    print(f"\n{city}:")
    print(f"  Celle ben servite (≥3 servizi): {stats['well_served_cells']}")
    print(f"  Celle poco servite (<3 servizi): {stats['poorly_served_cells']}")
    print(f"  Percentuale ben servita: {stats['percentage_well_served']:.1f}%")

## 11. Configurazione Mappa KeplerGL 3D

Configura la visualizzazione 3D dove l'altezza degli esagoni rappresenta la densità di servizi.

In [ ]:
# Configurazione avanzata per visualizzazione 3D
kepler_config = {
    'version': 'v1',
    'config': {
        'visState': {
            'filters': [],
            'layers': [
                {
                    'id': 'h3-3d-layer',
                    'type': 'hexagonId',
                    'config': {
                        'dataId': 'services',
                        'label': 'Service Density 3D',
                        'color': [255, 203, 153],
                        'columns': {'hex_id': 'h3_index'},
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'colorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
                            },
                            'coverage': 1,
                            'enable3d': True,
                            'sizeRange': [0, 500],
                            'coverageRange': [0, 1],
                            'elevationScale': 5,
                            'enableElevationZoomFactor': True
                        },
                        'hidden': False,
                        'textLabel': [
                            {
                                'field': None,
                                'color': [255, 255, 255],
                                'size': 18,
                                'offset': [0, 0],
                                'anchor': 'start',
                                'alignment': 'center'
                            }
                        ]
                    },
                    'visualChannels': {
                        'colorField': {'name': 'service_count', 'type': 'integer'},
                        'colorScale': 'quantile',
                        'sizeField': {'name': 'service_count', 'type': 'integer'},
                        'sizeScale': 'linear',
                        'coverageField': None,
                        'coverageScale': 'linear'
                    }
                }
            ],
            'interactionConfig': {
                'tooltip': {
                    'fieldsToShow': {
                        'services': [
                            {'name': 'city', 'format': None},
                            {'name': 'service_count', 'format': None},
                            {'name': 'Health', 'format': None},
                            {'name': 'Education', 'format': None},
                            {'name': 'Food', 'format': None},
                            {'name': 'Food Retail', 'format': None},
                            {'name': 'Retail', 'format': None},
                            {'name': 'Services', 'format': None},
                            {'name': 'Financial', 'format': None},
                            {'name': 'Public Services', 'format': None},
                            {'name': 'Security', 'format': None},
                            {'name': 'Sports', 'format': None},
                            {'name': 'Recreation', 'format': None},
                            {'name': 'Culture', 'format': None},
                            {'name': 'Transportation', 'format': None},
                            {'name': 'Tourism', 'format': None}
                        ]
                    },
                    'compareMode': False,
                    'compareType': 'absolute',
                    'enabled': True
                },
                'brush': {'size': 0.5, 'enabled': False},
                'geocoder': {'enabled': False},
                'coordinate': {'enabled': False}
            },
            'layerBlending': 'normal',
            'splitMaps': [],
            'animationConfig': {'currentTime': None, 'speed': 1}
        },
        'mapState': {
            'bearing': 24,
            'dragRotate': True,
            'latitude': 43.7,
            'longitude': 9.5,
            'pitch': 50,
            'zoom': 8,
            'isSplit': False
        },
        'mapStyle': {
            'styleType': 'dark',
            'topLayerGroups': {},
            'visibleLayerGroups': {
                'label': True,
                'road': True,
                'border': False,
                'building': True,
                'water': True,
                'land': True,
                '3d building': False
            },
            'threeDBuildingColor': [9.665468314072013, 17.18305478057247, 31.1442867897876],
            'mapStyles': {}
        }
    }
}

print("Configurazione KeplerGL 3D creata!")


## 12. Creazione Mappa Interattiva 3D

**Controlli mappa:**
- Shift + trascina mouse: ruota la vista
- Scroll: zoom
- Trascina: muovi la mappa

In [ ]:
# Inizializza mappa con configurazione
map_3d = KeplerGl(height=700, config=kepler_config)

# Aggiungi dati
map_3d.add_data(data=df_final, name='services')

print("Mappa 3D creata! Gli esagoni sono alti in base al numero di servizi.")
print("\nControlli mappa:")
print("- Tieni premuto Shift + trascina mouse: ruota la vista")
print("- Scroll: zoom")
print("- Trascina: muovi la mappa")

# Mostra mappa
map_3d

## 13. Mappa Comparativa Split (Opzionale)

Visualizzazione affiancata delle due città per confronto diretto.

In [ ]:
# Configurazione per split map (confronto città)
kepler_config_split = {
    'version': 'v1',
    'config': {
        'visState': {
            'filters': [
                {
                    'dataId': ['services'],
                    'id': 'city-filter',
                    'name': ['city'],
                    'type': 'multiSelect',
                    'value': [],
                    'enlarged': False,
                    'plotType': 'histogram',
                    'animationWindow': 'free',
                    'yAxis': None,
                    'speed': 1
                }
            ],
            'layers': [
                {
                    'id': 'h3-split',
                    'type': 'hexagonId',
                    'config': {
                        'dataId': 'services',
                        'label': 'Services by City',
                        'color': [18, 147, 154],
                        'columns': {'hex_id': 'h3_index'},
                        'isVisible': True,
                        'visConfig': {
                            'opacity': 0.8,
                            'colorRange': {
                                'name': 'Global Warming',
                                'type': 'sequential',
                                'category': 'Uber',
                                'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']
                            },
                            'coverage': 1,
                            'enable3d': True,
                            'sizeRange': [0, 500],
                            'elevationScale': 5
                        }
                    },
                    'visualChannels': {
                        'colorField': {'name': 'service_count', 'type': 'integer'},
                        'colorScale': 'quantile',
                        'sizeField': {'name': 'service_count', 'type': 'integer'},
                        'sizeScale': 'linear'
                    }
                }
            ],
            'splitMaps': [
                {'layers': {'h3-split': True}},
                {'layers': {'h3-split': True}}
            ]
        },
        'mapState': {
            'bearing': 0,
            'latitude': 43.7,
            'longitude': 9.5,
            'pitch': 40,
            'zoom': 7,
            'isSplit': True
        }
    }
}

map_split = KeplerGl(height=700, config=kepler_config_split)
map_split.add_data(data=df_final, name='services')

print("Mappa split creata per confronto diretto tra città!")
map_split

## 14. Salvataggio Mappe e Dati

In [ ]:
# Salva mappa 3D
map_3d.save_to_html(file_name='services_map_3d.html')
print("✓ Mappa 3D salvata: services_map_3d.html")

# Salva mappa split
map_split.save_to_html(file_name='services_map_split.html')
print("✓ Mappa split salvata: services_map_split.html")

# Salva anche i dati processati
df_final.to_csv('services_h3_aggregated.csv', index=False)
print("✓ Dati aggregati salvati: services_h3_aggregated.csv")

## 15. Download Files (Google Colab)

In [ ]:
from google.colab import files

# Download mappe
# files.download('services_map_3d.html')
files.download('services_map_split.html')
#files.download('services_h3_aggregated.csv')

In [ ]:
from IPython.display import IFrame
# IFrame(src='services_map_3d.html', width='100%', height=700)

## 16. Report Finale

Riepilogo completo dell'analisi e dei risultati ottenuti.

In [ ]:
print("="*70)
print(" "*20 + "REPORT FINALE")
print("="*70)

print(f"\nDATI PROCESSATI:")
print(f"  • POIs totali scaricati: {len(gdf_pois)}")
print(f"  • Celle H3 uniche (resolution 9): {len(df_final)}")
print(f"  • Città analizzate: {', '.join(df_final['city'].unique())}")

print(f"\nCONFRONTO CITTÀ:")
for city in df_final['city'].unique():
    city_data = df_final[df_final['city'] == city]
    print(f"\n  {city}:")
    print(f"    - Celle totali: {len(city_data)}")
    print(f"    - Servizi totali: {city_data['service_count'].sum()}")
    print(f"    - Densità media: {city_data['service_count'].mean():.2f} servizi/cella")
    print(f"    - Accessibilità: {accessibility[city]['percentage_well_served']:.1f}% celle ben servite")

print(f"\nFILE GENERATI:")
print(f"  • services_map_3d.html - Mappa 3D interattiva")
print(f"  • services_map_split.html - Mappa comparativa")
print(f"  • services_h3_aggregated.csv - Dati aggregati")

print(f"\nVISUALIZZAZIONE 3D:")
print(f"  L'altezza degli esagoni rappresenta il numero di servizi disponibili")
print(f"  nella cella, permettendo di identificare immediatamente le aree")
print(f"  con maggiore e minore densità di servizi essenziali.")

print("\n" + "="*70)